# install and import all requirements

In [ ]:
! pip install selenium beautifulsoup4


In [ ]:
!apt-get update
!apt install chromium-chromedriver


In [ ]:
! pip install google-colab-selenium

In [ ]:
! pip install pypdf2

In [ ]:
! pip install langchain

In [ ]:
! pip install langchain_openai

In [ ]:
! pip install transformers pandas torch regex


In [ ]:
! pip install -U langchain-community

In [ ]:
! pip install chromadb

In [ ]:
import google_colab_selenium as gs

driver = gs.Chrome()
# Your code to interact with the driver here
# ...
#driver.quit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import requests
import os
import re
import PyPDF2
import glob
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
import spacy
import re
import pandas as pd
from langchain.vectorstores import Chroma
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings

# finding links for opportunites based on the filters

In [ ]:

search_url = "https://bcbid.gov.bc.ca/page.aspx/en/rfp/request_browse_public"  # Replace with the actual search engine URL
driver.get(search_url)

# Wait for the page to load (adjust based on the site loading time)
time.sleep(5)

# Find the search input field (adjust the XPath or class based on the site's HTML)
dropdown = driver.find_element(By.CSS_SELECTOR, "div[data-selector='body_x_selRtgrouCode']")
dropdown.click()

# Wait for the dropdown options to appear
time.sleep(1)

# Select "Invitation to Bid"
invitation_to_bid = driver.find_element(By.CSS_SELECTOR, "li[data-value='itb']")
invitation_to_bid.click()

# Select "Invitation to Quote"
invitation_to_quote = driver.find_element(By.CSS_SELECTOR, "li[data-value='itq']")
invitation_to_quote.click()



request_for_proposal = driver.find_element(By.CSS_SELECTOR, 'li[data-value="rfp"]')
request_for_proposal.click()

request_for_tenders = driver.find_element(By.CSS_SELECTOR, 'li[data-value="Rft"]')
request_for_tenders.click()

invitation_to_offer = driver.find_element(By.CSS_SELECTOR, 'li[data-value="io"]')
invitation_to_offer.click()

# Wait a bit for the selection to register
time.sleep(1)

# Proceed to submit or perform the search
search_button = driver.find_element(By.ID, 'body_x_prxFilterBar_x_cmdSearchBtn')
  # Adjust the selector to match the actual search button
search_button.click()

# Wait for results to load
time.sleep(5)

# Extract the page source after the results are loaded
page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html.parser')

# Finding all <a> tags and extracting 'href' attributes
hrefs = [a['href'] for a in soup.find_all('a', href=True)]

# Printing the results
for href in hrefs:
    links.append("https://bcbid.gov.bc.ca/" + href)

# Close the WebDriver
driver.quit()

# Process or print the page source
print(page_source)


In [ ]:
links

['https://bcbid.gov.bc.ca/#divLinkTarget',
 'https://bcbid.gov.bc.ca//?lang=en',
 'https://bcbid.gov.bc.ca//page.aspx/en/rfp/request_browse_public',
 'https://bcbid.gov.bc.ca//page.aspx/en/ctr/contract_browse_public',
 'https://bcbid.gov.bc.ca//page.aspx/en/rfp/unverified_bids_browse_public',
 'https://bcbid.gov.bc.ca//page.aspx/en/ctn/links_public_browse',
 'https://bcbid.gov.bc.ca/#',
 'https://bcbid.gov.bc.ca/https://lbip.outcome-plus.com/Advertisement/AdvertisementLanding',
 'https://bcbid.gov.bc.ca/https://fesims.outcome-plus.com/Advertisement/AdvertisementLanding',
 'https://bcbid.gov.bc.ca//page.aspx/en/bpm/process_manage_extranet/204554',
 'https://bcbid.gov.bc.ca//page.aspx/en/bpm/process_manage_extranet/204552',
 'https://bcbid.gov.bc.ca//page.aspx/en/bpm/process_manage_extranet/204550',
 'https://bcbid.gov.bc.ca//page.aspx/en/bpm/process_manage_extranet/204548',
 'https://bcbid.gov.bc.ca//page.aspx/en/bpm/process_manage_extranet/204546',
 'https://bcbid.gov.bc.ca//page.aspx/

# going through all links and find download links for documents

In [ ]:

def download_file(url, folder="downloaded_files"):
    """Downloads the file from the specified URL into the designated folder."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful

        # Parse filename from URL or set a default
        filename = url.split('/')[-1] or 'downloaded_file.pdf'
        save_path = os.path.join(folder, filename)

        # Create the directory if it does not exist
        if not os.path.exists(folder):
            os.makedirs(folder)

        # Save the file
        with open(save_path, 'wb') as f:
            f.write(response.content)

        print(f"File downloaded successfully: {save_path}")
    except requests.RequestException as e:
        print(f"Error downloading the file: {e}")

# URL to scrape
for url in links:
  # Send HTTP request
  print(url)
  response = requests.get(url)
  response.raise_for_status()  # Check for HTTP issues

  # Parse the HTML
  soup = BeautifulSoup(response.text, 'html.parser')

  # Extract the download link (assuming the structure is consistent with the provided snippet)
  a_tag = soup.find('a', href=True, id="body_x_tabc_rfp_ext_prxrfp_ext_x_prxDoc_x_grid_grd__ctl2_files_x_btnDownload_C2A03403-3B4F-45EA-B9BD-700C8E4CB7DC")
  if a_tag:
      download_link = "https://bcbid.gov.bc.ca" + a_tag['href']
      print(f"Found download link: {download_link}")
      download_file(download_link)
  else:
      print("No download link found.")
# body_x_tabc_rfp_ext_prxrfp_ext_x_phcDoc_field_content
# https://bcbid.gov.bc.ca//page.aspx/en/bpm/process_manage_extranet/204493

https://bcbid.gov.bc.ca/#divLinkTarget
No download link found.
https://bcbid.gov.bc.ca//?lang=en
No download link found.
https://bcbid.gov.bc.ca//page.aspx/en/rfp/request_browse_public
No download link found.
https://bcbid.gov.bc.ca//page.aspx/en/ctr/contract_browse_public
No download link found.
https://bcbid.gov.bc.ca//page.aspx/en/rfp/unverified_bids_browse_public
No download link found.
https://bcbid.gov.bc.ca//page.aspx/en/ctn/links_public_browse
No download link found.
https://bcbid.gov.bc.ca/#
No download link found.
https://bcbid.gov.bc.ca/https://lbip.outcome-plus.com/Advertisement/AdvertisementLanding
No download link found.
https://bcbid.gov.bc.ca/https://fesims.outcome-plus.com/Advertisement/AdvertisementLanding
No download link found.
https://bcbid.gov.bc.ca//page.aspx/en/bpm/process_manage_extranet/204554
No download link found.
https://bcbid.gov.bc.ca//page.aspx/en/bpm/process_manage_extranet/204552
No download link found.
https://bcbid.gov.bc.ca//page.aspx/en/bpm/proces

In [ ]:
for url in links:
  response = requests.get(url)

# Check if the request was successful
  if response.status_code == 200:
      # Parse the page content using BeautifulSoup
      soup = BeautifulSoup(response.content, 'html.parser')

      # Define the regex pattern to match the file download links
      pattern = re.compile(r'/bare\.aspx/en/fil/download_public/.*')

      # Find all <a> tags with href attributes matching the pattern
      file_links = []
      for a_tag in soup.find_all('a', href=pattern):
          file_link = a_tag['href']

          # If the link is relative, prepend the domain
          if file_link.startswith('/'):
              file_link = f"https://bcbid.gov.bc.ca{file_link}"

          file_links.append(file_link)
          print(f"Found download link: {file_link}")

      # If you want to download each file
      for i, link in enumerate(file_links):
          file_name = f"file_{i+1}.pdf"  # You can modify this to extract the actual file name from the link
          file_response = requests.get(link)

          if file_response.status_code == 200:
              with open(file_name, 'wb') as f:
                  f.write(file_response.content)
              print(f"Downloaded {file_name}")
          else:
              print(f"Failed to download file from {link}")
  else:
      print(f"Failed to fetch the webpage. Status code: {response.status_code}")

Found download link: https://bcbid.gov.bc.ca/bare.aspx/en/fil/download_public/40c90db1-4821-48ec-b5aa-5e9749890473?file_context%5BcontainerUrl%5D=/rfp/rfp_info_extranet/204554/&file_context%5Brfp%5D=204554
Downloaded file_1.pdf
Found download link: https://bcbid.gov.bc.ca/bare.aspx/en/fil/download_public/19090b7f-7d33-43eb-b629-4d53128ce935?file_context%5BcontainerUrl%5D=/rfp/rfp_info_extranet/204552/&file_context%5Brfp%5D=204552
Found download link: https://bcbid.gov.bc.ca/bare.aspx/en/fil/download_public/55c90622-3fbb-4777-9d18-1cb2e0539af8?file_context%5BcontainerUrl%5D=/rfp/rfp_info_extranet/204552/&file_context%5Brfp%5D=204552
Downloaded file_1.pdf
Downloaded file_2.pdf
Found download link: https://bcbid.gov.bc.ca/bare.aspx/en/fil/download_public/0d2b7db1-2565-4c59-85d1-97673a9369bf?file_context%5BcontainerUrl%5D=/rfp/rfp_info_extranet/204538/&file_context%5Brfp%5D=204538
Found download link: https://bcbid.gov.bc.ca/bare.aspx/en/fil/download_public/16993272-a3d0-446c-9016-2b60d646

# find summary, category of each document, date of submission, name of the company or stackholders and value of the request per document

In [ ]:
OPENAI_API_KEY = ""

llm =  ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0.0, model = "gpt-4o-mini")

def generate_insight(content):
    retrieved_docs = content
    # context = "\n\n".join([doc[0].page_content for doc in retrieved_docs])

    ANSWER_PROMPT = ChatPromptTemplate.from_template("""
    You are given a document that is an RFX (Request for X) type such as bids, quotations, or requests. Your task is to extract the following information:

    1. **Value of the Request (in dollar terms)**: Identify and extract the total value of the request or any cost estimation mentioned in the document.
    2. **Category of the Document**: Determine whether the document is a Request for Proposal (RFP), Request for Quotation (RFQ), Invitation to Bid, Invitation to tender ,or Invitation to Quote.
    3. **Summary of the Document**: Provide a concise summary of the key details, including the purpose of the request, the main requirements, and any important deadlines or deliverables.
    4. ** stakeholders and companies**: Extract the name of the company or companies, and stackholders and persons mentioned in the document.
    5. **date of submittion**: Extract the date the document was submitted.
    6. **Helpful Insights**: Extract any other relevant information or insights that might be helpful, such as evaluation criteria, project timelines, or any special instructions.


    context: {context}
        Answer:
    """
    )

    chain = (
        {"context": RunnablePassthrough()}
        | ANSWER_PROMPT
        | llm
        | StrOutputParser()
    )

    ans = chain.invoke({"context":retrieved_docs})
    return ans

In [20]:
# /content/*
pdf_directory = '/content/'
pdf_files = glob.glob(os.path.join(pdf_directory, '*.pdf'))
c = 0
# Loop through each PDF and open it
for pdf_file in pdf_files:
    try:
        with open(pdf_file, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            num_pages = len(reader.pages)
            print(f'File: {pdf_file} - Pages: {num_pages}')
            # Example of reading the first page
            first_page = reader.pages[0].extract_text()
            # print(first_page)  # Do something with the text
            c +=1
            info = reader.metadata
            num_pages = len(reader.pages)
            content = ""
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                content += page.extract_text()
            # title = info.title
            # print(title)
            print(generate_insight(content))
            print("#####################################################################")
            print()
    except PyPDF2.errors.PdfReadError as e:
        print(f"Error reading {pdf_file}: {e}")
print(c)

File: /content/SD25TKN048_g_Schedule_B_Payment.pdf - Pages: 4
1. **Value of the Request (in dollar terms)**: The document does not specify a total value for the request, as the fields for the rates and total fees are left blank (e.g., "$_________"). It mentions that the total fees payable will not exceed a certain amount, but this amount is also not provided.

2. **Category of the Document**: The document appears to be a **Contract Payment Schedule** related to a service agreement, rather than a traditional RFP, RFQ, or Invitation to Bid. It outlines payment terms for services rendered under a specific contract.

3. **Summary of the Document**: 
   - The document outlines the payment schedule for services related to the "Saunier Fire Salvage Layout" project.
   - It specifies various unit rates for different types of work (e.g., field crew days, senior field days, office days, professional assessments, etc.) but leaves the actual rates blank.
   - The document emphasizes that the quant

# finding the organization, dollar request and people using spacy

In [21]:
nlp = spacy.load("en_core_web_sm")

def details(doc_text):
    doc = nlp(doc_text)
    for ent in doc.ents:
        if ent.label_ in ["ORG", "GPE", "PERSON"]:
            print(f"{ent.label_}: {ent.text}")

    # Regex for dollar amounts and dates
    dollar_amounts = re.findall(r'\$\d+(?:,\d{3})*(?:\.\d{2})?', doc_text)
    dates = re.findall(r'\b(?:\d{1,2}[-/thstndrd\s]*)?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)?\s?\d{1,2}[-/,\s]*\d{2,4}\b', doc_text)

    print("Dollar Amounts:", dollar_amounts)
    print("Dates:", dates)

In [30]:
docs = []
for pdf_file in pdf_files:
    try:
        with open(pdf_file, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            num_pages = len(reader.pages)
            first_page = reader.pages[0].extract_text()
            c +=1
            info = reader.metadata
            num_pages = len(reader.pages)
            content = ""
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                content += page.extract_text()
            # title = info.title
            # print(title)
            print(details(content))
            print("#####################################################################")
            print()
            docs.append(content)
    except PyPDF2.errors.PdfReadError as e:
        print(f"Error reading {pdf_file}: {e}")


Streaming output truncated to the last 5000 lines.
ORG: BC Hydro
ORG: Contracts
ORG: the BC Bid Address  
Notwithstanding Section
ORG: the BC Bid Address
ORG: BC Hydro
ORG: BC Hydro
ORG: BC Bid
GPE: Proponent
ORG: Conflict of Interest
ORG: Unfair Advantage
ORG: BC Hydr
ORG: BC Hydro
ORG: bee n
ORG: BC Hydro
ORG: BC Hydro ’s
GPE: Proponent
ORG: BC Hydro
ORG: BC Hydro
ORG: BC Hydro
GPE: Proponent
GPE: Proponent
ORG: the Contact Person
ORG: BC Hydro
ORG: BC Hydro ’s
ORG: BC Hydro
ORG: BC Hydro
ORG: BC Hydro
GPE: Proponent
GPE: Proponent
ORG: a Restricted Party
GPE: Proponent
GPE: Proponent
ORG: the Contact Person
ORG: BC Hydro
ORG: BC Hydro
PERSON: Proposal
ORG: Section
ORG: BC Hydro
ORG: BC Hydro
ORG: BC Hydro
ORG: Freedom of Information and Protection of Privacy Act
GPE: British Columbia
ORG: BC Hydro
GPE: Proposals
ORG: Personal Information
ORG: BC Hydro
GPE: Proponent
ORG: BC Hydro
ORG: the Personal Information
ORG: BC Hydro
ORG: BC Hydro
GPE: Proposals
ORG: BC Hydro
PERSON: Proposal


In [31]:
len(docs)

33

# finding the same analysis using  bert classifier

In [25]:
# Load pre-trained BERT model for NER
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
def extract_entities(text):
    # Apply BERT-based NER
    entities = ner_pipeline(text)

    # Filter entities
    companies = [ent['word'] for ent in entities if ent['entity_group'] == 'ORG']
    persons = [ent['word'] for ent in entities if ent['entity_group'] == 'PER']
    locations = [ent['word'] for ent in entities if ent['entity_group'] == 'LOC']

    return {"companies": companies, "persons": persons, "locations": locations}


In [27]:
def extract_amounts_and_dates(text):
    # Regex for dollar amounts
    dollar_amounts = re.findall(r'\$\d+(?:,\d{3})*(?:\.\d{2})?', text)

    # Regex for dates
    dates = re.findall(r'\b(?:\d{1,2}[-/thstndrd\s]*)?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)?\s?\d{1,2}[-/,\s]*\d{2,4}\b', text)

    return {"dollar_amounts": dollar_amounts, "dates": dates}


In [28]:
def process_documents(documents):
    data = []

    for doc_text in documents:
        # Extract named entities using BERT NER
        entities = extract_entities(doc_text)

        # Extract dollar amounts and dates using regex
        amounts_and_dates = extract_amounts_and_dates(doc_text)

        # Extracted information for this document
        doc_data = {
            "company_names": entities["companies"],
            "stakeholders": entities["persons"],
            "locations": entities["locations"],
            "dollar_amounts": amounts_and_dates["dollar_amounts"],
            "dates": amounts_and_dates["dates"]
        }
        data.append(doc_data)

    # Create DataFrame
    df = pd.DataFrame(data)
    return df


In [32]:
df = process_documents(docs)


In [33]:
df.head()

,company_names,stakeholders,locations,dollar_amounts,dates
0,"[CSNR, Ministry of Forests, ##er Fire Sal, Lay...",[],[],"[$180.90, $165.60]","[May 2023, 10005, May 2023, 180, 165, May 2023..."
1,"[Con, Natural Resource Mini, ##ries, CO, ##BIA...","[Wolfgang Beck, Wolfgang, Beck]","[##UM, Okanagan, Columbia, Avenue Vernon, Brit...",[],"[September 9, 2022, 10005, 20 24, 2501 14, 778..."
2,"[BCTS, Ministry of Forests BC, Re, ##ibility B...",[],"[British Columbia, British Columbia, British C...","[$150, $5]","[ 2022, 150, 2022, 2022]"
3,"[Fac, and Recreation, GRAND, RDK, RDK, RDK, ##D]",[Paul Keys],"[G, ##ND, GRAND, G, ##ND, Regional District of...","[$5,000,000, $10,000]","[September 17, 2024, November 7, 2024, 2020, ..."
4,"[Sandblast Cabinet, ##les, BC Bid, BC, BC B, B...","[Elizabeth Robles, Elizabeth, r]",[],[],"[10370515, 21098, August 23, 2023, 21098, May..."


# classifying the documents

In [37]:
def classify_document(text):
    # Define keywords for each category
    rfp_keywords = ["request for proposal", "rfp", "statement of work", "scope of work", "proposal submission"]
    rfq_keywords = ["request for quotation", "rfq", "quotation", "quote submission"]
    bid_keywords = ["invitation to bid", "bid submission", "sealed bid", "bid process"]
    tender_keywords = ["invitation to tender", "tender process", "tender submission", "sealed tender"]
    quote_keywords = ["invitation to quote", "quote submission", "quotation"]


    # Lowercase the document for case-insensitive matching
    text_lower = text.lower()

    # Check for keywords and classify accordingly
    if any(keyword in text_lower for keyword in rfp_keywords):
        return "RFP (Request for Proposal)"
    elif any(keyword in text_lower for keyword in rfq_keywords):
        return "RFQ (Request for Quotation)"
    elif any(keyword in text_lower for keyword in bid_keywords):
        return "Invitation to Bid"
    elif any(keyword in text_lower for keyword in tender_keywords):
        return "Invitation to Tender"
    elif any(keyword in text_lower for keyword in quote_keywords):
        return "Invitation to Quote"
    else:
        return "other documents"


In [38]:
def classify_documents(documents):
    classifications = []
    for doc_text in documents:
        classification = classify_document(doc_text)
        classifications.append(classification)
    return classifications


# Classify documents
classifications = classify_documents(docs)
for i, classification in enumerate(classifications):
    print(f"Document {i + 1}: {classification}")


Document 1: other documents
Document 2: other documents
Document 3: Invitation to Tender
Document 4: RFP (Request for Proposal)
Document 5: Invitation to Bid
Document 6: Invitation to Tender
Document 7: RFP (Request for Proposal)
Document 8: other documents
Document 9: other documents
Document 10: RFP (Request for Proposal)
Document 11: other documents
Document 12: other documents
Document 13: other documents
Document 14: other documents
Document 15: RFP (Request for Proposal)
Document 16: other documents
Document 17: RFP (Request for Proposal)
Document 18: other documents
Document 19: other documents
Document 20: other documents
Document 21: other documents
Document 22: other documents
Document 23: RFP (Request for Proposal)
Document 24: RFP (Request for Proposal)
Document 25: RFQ (Request for Quotation)
Document 26: RFP (Request for Proposal)
Document 27: other documents
Document 28: Invitation to Bid
Document 29: other documents
Document 30: RFP (Request for Proposal)
Document 31: o

# implementing simple rag method for question, answering
## for chuncking I used RecursiveCharacterTextSplitter, for vecotr database I used chroma and gpt 4 for responses.

In [39]:
def splitting_text_recursive(documents, chunk_size=1000, chunk_overlap=200, add_start_index=True):

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, add_start_index=add_start_index
    )
    all_splits = text_splitter.split_documents(documents)

    return all_splits

In [42]:

def retrieve_from_chroma(vector_chroma, query_vector, metric='cosine', top_k=5, collection_name='default_collection'):

    if metric not in ['cosine', 'euclidean', 'manhattan']:
        raise ValueError("Unsupported metric. Choose from 'cosine', 'euclidean', 'manhattan'.")

    metric_map = {
        'cosine': 'cosine',
        'euclidean': 'l2',
        'manhattan': 'l1'
    }

    results = vector_chroma.similarity_search_with_score(
        query=query_vector,
        k=top_k
    )

    return results

In [44]:
def generate_response(question, docs):
        retrieved_docs = docs
        context = "\n\n".join([doc[0].page_content for doc in retrieved_docs])

        ANSWER_PROMPT = ChatPromptTemplate.from_template(
            """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible.

            context: {context}
            Question: "{question}"
            Answer:
            """
        )

        chain = (
            {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
            | ANSWER_PROMPT
            | llm
            | StrOutputParser()
        )

        ans = chain.invoke({"context":context, "question":question})
        return ans

In [76]:

splits = splitting_text_recursive([docs[0][0]])
openai_api_key = OPENAI_API_KEY

vectorstore = Chroma.from_documents(documents=splits,
                                        embedding= OpenAIEmbeddings(openai_api_key=openai_api_key))

docs = retrieve_from_chroma(vectorstore, query_vector="what is law?")
#  print(docs[0][0].page_content)
rag_generator = generate_response("what is law?", docs)


In [75]:
docs[0][0]

Document(metadata={'start_index': 7326}, page_content='(a) your legal name, address, the date and the peri od of time which the invoice applies (“Billing \nPeriod”), the contract number, and a statement number for identification; \n(b) the calculation of all fees claimed under this  Agreement for the Billing Period, with hours, dates, \nrates, and name(s) of persons providing the Se rvices, a description of specific services/works \ncompleted during the Billing Period, including a declaration that the Services have been \ncompleted; \n(c) and where expenses are to be paid under this Agreement, a chronological listing, in reasonable \ndetail and with dates, of all expenses claimed by  you under this Agreement for the Billing Period \nwith receipts or copies of receipts, where applicable, attached; \n(d)  if you are claiming reimbursement of any GS T or other applicable taxes paid or payable by you \nin relation to those expenses, a description of any credits, rebates, refunds, or remiss

In [77]:
print(rag_generator)

Law is a system of rules and guidelines, typically enforced through social institutions to govern behavior. It serves as a framework for regulating the conduct of individuals and organizations within a society. Laws are created by legislative bodies, interpreted by courts, and enforced by various governmental agencies. 

The primary purposes of law include maintaining order, protecting individual rights and liberties, resolving disputes, and promoting social justice. Laws can be categorized into various types, such as criminal law, which deals with offenses against the state; civil law, which addresses disputes between individuals or organizations; and administrative law, which governs the activities of governmental agencies.

Legal systems vary widely across different countries and cultures, influenced by historical, social, and political factors. Common legal traditions include common law, which is based on judicial precedents, and civil law, which is based on codified statutes.

In 